## Convert an image of a face to a network

Code adapted from [this blog post](https://vedransekara.github.io/) by Vedran Sekara.

**The original code** takes a solid black and white image and *converts it to a network*, by sampling points randomly inside the black region and connects the points to their nearest neighbors.

**This code** does almost the same, except the sampling is biased by the greyscale pixel intensity, and the number of links for each sampled point scales with the darkness of the region in which it was sampled. It works well for faces.

**Imports and functions**

In [13]:
import sys
sys.path.append("/Users/ulfaslak/anaconda2/lib/python2.7/site-packages")
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
import imageio
import os

In [14]:
def rgb2gray(rgb):
    """Transform color to grayscale."""
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

### Static image

**Parameters**

The input file has to be in either PNG or JPEG format. For the best results, use PNG and make sure the background is completely transparent. You can, for example, do this with [lunapic](https://www298.lunapic.com/editor/?action=transparent) or in Apple Keynote using the *Instant Alpha* function.

In [32]:
filename = "sage"
nsamples = 10000   # Number of nodes
k_max = 10         # The maximum number of links per node
k_min = 3          # The maximum number of links per node
contrast = 3       # Increase to when skin is darker (2-3 is good)

**Sample from image**

Depending on skin-color, the clear skin areas will typically be brigter and have higher intensity than the features of the face. For faces though, we want dark regions to have lots of links in the network. Therefore, we invert the image, to highlight darker regions with lots of nodes and links. Subsequently we sample points from the data, and bias the sampling by the intensities exponentiated by `contrast`.

In [33]:
# Load and process data
img = plt.imread('%s.png' % filename)
data = rgb2gray(img)
data = (1 - data / data.max()) * (img[:, :, 3] != 0)  # Invert and zero out background
data = data.T                                         # Transpose coordinate rotation
print(data.shape)
x_norm, y_norm = map(float, data.shape)
r = x_norm / y_norm

# Sample
p_map = data**contrast / np.sum(data**contrast)        # Probability of each pixel
ij = np.random.choice(
    np.arange(0, data.shape[0] * data.shape[1]),       # Flattened indices
    size=nsamples, p=p_map.reshape(-1)
)

# Sampled coordinates
X = np.array(list(zip(ij / data.shape[1], ij % data.shape[1]))) + np.random.random(size=(nsamples, 2))-0.5  # small jitter to rid the gridder

# Nearest neighbors
tree = cKDTree(X)

(828, 903)


**Plot**

First we define all the links.

In [36]:
# Create lists for position of links
x, y = [], []

# Go through each node and construct links 
for pt in X:
    k = int(data[int(round(pt[0])-1), int(round(pt[1])-1)] / np.max(data) * (k_max - k_min) + k_min)
    dist, ind = tree.query(pt, k=k)
    for kneigh in ind[1:]:
        x.append([pt[0], X[kneigh][0]])
        y.append([pt[1], X[kneigh][1]])

And then we draw and save them.

In [37]:
# Construct figure
plt.figure(figsize=(4*r, 4), frameon=False)
ax = plt.subplot(111)

plt.plot(np.array(x).T, np.array(y).T, color='#282828', lw=0.4, alpha=0.4, zorder=2) 

plt.axis('off')
plt.ylim(y_norm, 0)
plt.xlim(0, x_norm)

#ax.set_ylim(ax.get_xlim()[::-1])
plt.savefig('%s_bot%d_pow%.01fsampling_kmin%d_kmax%d.png' % (filename, nsamples, contrast, k_min, k_max), dpi=250, pad=0.0, bbox_inches='tight')
plt.close()

### Evolving gif

**Parameters**

In [38]:
filename = "sage"
nsamples_min = 100     # Number of nodes in first frame
nsamples_max = 10000   # Number of nodes in last frame
nframes = 50           # Number of frames
k_max = 10             # The maximum number of links per node
k_min = 3              # The maximum number of links per node
contrast = 3           # Increase to when skin is darker (2-3 is good)

**Process in loop**

Essentially, we just do the same as above, but now for a steadily increasing number of samples. In each iteration we draw a fixed number of new samples, and add them to the existing ones, then compute nearest neighbors and draw links for all points again. If you move `x, y = [], []` out of the loop and iterate over `X_` and not `X` in line 36, the process is marginally faster and the result will be slighly different. In the end we stitch all the frames together and create a gif.

In [40]:
# Load and process data
img = plt.imread('%s.png' % filename)
data = rgb2gray(img)
data = (1 - data / data.max()) * (img[:, :, 3] != 0)  # Invert and zero out background
data = data.T                                         # Transpose coordinate rotation
x_norm, y_norm = map(float, data.shape)
r = x_norm / y_norm

# Sample
p_map = data**contrast / np.sum(data**contrast)        # Probability of each pixel

images = []
X = np.empty(shape=(0, 2))
for nsamples in map(int, np.logspace(np.log10(nsamples_min), np.log10(nsamples_max), nframes)):
    
    ij = np.random.choice(
        np.arange(0, data.shape[0] * data.shape[1]),       # Flattened indices
        size=(nsamples-X.shape[0]), p=p_map.reshape(-1)
    )

    # Sampled coordinates
    X_ = np.array(list(zip(
        ij / data.shape[1],                          # x coordinates
        ij % data.shape[1]                           # y coordinates
    ))) + (np.random.random(size=(nsamples-X.shape[0], 2))-0.5)  # small jitter to rid the gridder
    
    X = np.vstack([X, X_])
    
    outfilename = '%s_bot%d_pow%.01fsampling_kmin%d_kmax%d.png' % (filename, X.shape[0], contrast, k_min, k_max)
    
    if outfilename in os.listdir("."):
        print(nsamples, end=",")
        continue

    # Nearest neighbors
    tree = cKDTree(X)
    
    # Create lists for position of links
    x, y = [], []

    # Go through each node and construct links 
    for pt in X:
        k = int(data[int(round(pt[0])-1), int(round(pt[1])-1)] / np.max(data) * (k_max - k_min) + k_min)
        dist, ind = tree.query(pt, k=k)
        for kneigh in ind[1:]:
            x.append([pt[0], X[kneigh][0]])
            y.append([pt[1], X[kneigh][1]])
            
    # Construct figure (this is the part that takes the most time)
    plt.figure(figsize=(4*r, 4), frameon=False)
    ax = plt.subplot(111)

    plt.plot(np.array(x).T, np.array(y).T, color='#282828', lw=0.4, alpha=0.4, zorder=2) 

    plt.axis('off')
    plt.ylim(y_norm, 0)
    plt.xlim(0, x_norm)

    plt.savefig(outfilename, dpi=250, pad=0.0, bbox_inches='tight')
    plt.close()
    images.append(imageio.imread(
        outfilename
    ))
    
    print(nsamples, end=",")

# Render array of images as one gif
imageio.mimsave(
    '%s_bot%d_pow%.01fsampling_kmin%d_kmax%d_nsamples_%d_%d_nframes_%d.gif' % \
    (filename, X.shape[0], contrast, k_min, k_max, nsamples_min, nsamples_max, nframes),
    images
)

100,109,120,132,145,159,175,193,212,232,255,281,308,339,372,409,449,494,542,596,655,719,790,868,954,1048,1151,1264,1389,1526,1676,1842,2023,2222,2442,2682,2947,3237,3556,3906,4291,4714,5179,5689,6250,6866,7543,8286,9102,10000,